<a href="https://colab.research.google.com/github/YuehMintTai/RPython/blob/main/Class06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#三軍總醫院北投分院統計及實驗設計課程之六
## 2021/7/7
## ytai1123@gmail.com
### 使用方法:


1.   使用gmail帳號登入
2.   按"執行階段" -->"全部執行" 以執行全部內容, 若要個別執行可點選每格程式左方箭頭或按 Control + Enter 鍵執行。

In [ ]:
##0-1
!git clone https://github.com/YuehMintTai/RPython.git

Cloning into 'RPython'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (221/221), done.
remote: Total 223 (delta 127), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (223/223), 1010.21 KiB | 3.84 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [ ]:
##0-2
!pip install rpy2

In [ ]:
##0-3
%load_ext rpy2.ipython

In [ ]:
##6-1
%%R
myData<-read.csv('RPython/samples.csv')
tail(myData,1)

    SID 性別 年齡 入伍前職業 教育程度 婚姻狀況 皆無過去病史01 早產兒01
188   4    1   25         商        4        1              1        0
    頭部曾受傷01 發展遲緩01 注意力不足過動症01 癲癇01 癲癇服藥治療 癲癇服藥期間
188            0          0                  0      0            0            0
    軍種 軍階 役別 入伍至今_年 聽過自殺課程_次 求助心輔_次 求助精神科_次
188    1    1    2         0.5               1           0             2
    使用1995_次 使用24h專線_次 特殊狀況 父母婚姻狀態 自殺意念_bsrs6 B型肝炎01
188           0              0        4            4              4         0
    C型肝炎01 氣喘史01 過敏史01 心臟病史01 高血壓01 醣尿病01 甲狀腺01 類風濕01
188         0        1        1          0        0        0        1        0
    重大意外01 自殺意念01 透露父母 透露手足 透露好友 透露同儕 透露長官 透露心輔
188          1          1        0        0        0        0        0        0
    透露醫師 拒告父母 拒告手足 拒告好友 拒告同儕 拒告長官 拒告心輔 拒告醫師
188        0        1        1        1        1        1        1        1
    BSRS總分 BSRSR總分 過動症總分 Inattention Impulsivity opposition depression
188       20         5         18           9           

In [ ]:
##6-2
%%R
formula<-'自殺意念01~as.factor(性別)'
model1<-glm(formula, myData,family='binomial')
exp(cbind(OR=coef(model1), confint(model1)))

R[write to console]: Waiting for profiling to be done...



                        OR     2.5 %    97.5 %
(Intercept)      0.2514924 0.1682959 0.3648018
as.factor(性別)2 0.6915241 0.1928131 1.9540045


In [ ]:
##6-3
%%R
library(table2x2)
myData$自殺意念01<-as.integer(myData$自殺意念01)
myTable<-table(myData$自殺意念01,myData$性別)
rownames(myTable)<-c('0無自殺意念','1有自殺意念')
colnames(myTable)<-c('1男性','2女性')
table2x2(myTable, stats=c('table','rr'))

R[write to console]: Error in library(table2x2) : there is no package called ‘table2x2’




Error in library(table2x2) : there is no package called ‘table2x2’


RInterpreterError: ignored

In [ ]:
##6-3-1
%%R
with(myData, table(自殺意念01, 性別))

          性別
自殺意念01   1   2
         0 130  23
         1  31   4


In [ ]:
##6-4
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf
df=pd.read_csv('RPython/samples.csv')
formula='自殺意念01.astype(int)~C(性別)'
model3=smf.logit(formula=formula,data=df)
result=model3.fit()
result.summary()
print('OR={}, 95%CI={} to '.
      format(round(np.exp(result.params)[1],3),\
      round(np.exp(result.conf_int())[0][1],3)),\
      round(np.exp(result.conf_int())[1][1],3))


Optimization terminated successfully.
         Current function value: 0.479782
         Iterations 6
OR=0.729, 95%CI=0.235 to  2.262


In [ ]:
##6-4
import pandas as pd
import scipy.stats as stats
df=pd.read_csv('RPython/samples.csv')
df['自殺意念01']=df['自殺意念01'].astype(int)
myTable=pd.crosstab(df['自殺意念01'],df['性別'])
print(myTable)
(oddsratio, pvalue)=stats.fisher_exact(myTable, alternative='two-sided')
print('Odds Ratio:{}, p-value:{}'.format(round(oddsratio,2),round(pvalue,3)))


性別        1   2
自殺意念01         
0       130  23
1        31   4
Odds Ratio:0.73, p-value:0.79


In [ ]:
##6-5-1 MultiNomial logistic regression
%%R
with(myData, table(自殺意念_bsrs6,性別))


              性別
自殺意念_bsrs6   1   2
             0 118  21
             1  25   5
             2   9   1
             3   7   0
             4   2   0


In [ ]:
##6-6-1 Kappa -Fleiss’ Kappa
%%R
install.packages('irr')

In [ ]:
##6-6-2
%%R
library('irr')
data('diagnoses',package='irr')
#head(diagnoses)
diagnoses[1:3,1:6]

                   rater1                  rater2                  rater3
1             4. Neurosis             4. Neurosis             4. Neurosis
2 2. Personality Disorder 2. Personality Disorder 2. Personality Disorder
3 2. Personality Disorder        3. Schizophrenia        3. Schizophrenia
            rater4           rater5      rater6
1      4. Neurosis      4. Neurosis 4. Neurosis
2         5. Other         5. Other    5. Other
3 3. Schizophrenia 3. Schizophrenia    5. Other


In [ ]:
##6-6-3
%%R
library('irr')
myData<-diagnoses[,1:4]
myData
kappam.fleiss(myData, detail=TRUE)

 Fleiss' Kappa for m Raters

 Subjects = 30 
   Raters = 4 
    Kappa = 0.489 

        z = 13 
  p-value = 0 

                         Kappa      z p.value
1. Depression            0.377  5.056   0.000
2. Personality Disorder  0.410  5.497   0.000
3. Schizophrenia         0.596  7.996   0.000
4. Neurosis              0.348  4.663   0.000
5. Other                 0.812 10.900   0.000


In [ ]:
##6-6-4
%%R
# Here we create and display an artifical dataset,
# describing two traits coded by three raters:
scores <- data.frame(
	Trait1_A = c(1,0,2,1,1,1,0,2,1,1),
	Trait1_B = c(1,2,0,1,2,1,0,1,2,1),
	Trait1_C = c(2,2,2,1,1,1,0,1,2,1),
	Trait2_A = c(1,4,5,2,3,5,1,2,3,4),
	Trait2_B = c(2,5,2,2,4,5,1,3,1,4),
	Trait2_C = c(2,4,3,2,4,5,2,2,3,4)
	)
scores

# Retrieve Fleiss' kappa for Trait1 and Trait2,
# to evaluate inter-rater agreement between raters A, B and C:
#kappaFleiss(scores, nb_raters=3)
kappam.fleiss(scores)

 Fleiss' Kappa for m Raters

 Subjects = 10 
   Raters = 6 
    Kappa = 0.0581 

        z = 1.38 
  p-value = 0.169 


In [ ]:
%%R
install.packages('multinom')

In [ ]:
%%R
library(multinom)
formula<-'自殺意念_bsrs6~年齡+as.factor(性別)'
result<-multinom(fromula,myData)

In [ ]:
%%R
##install.packages('Publish')
remove.packages('Publish')

In [ ]:
%%R
library(Publish)
myTable<-data.frame(Case=c(12,34),Control=c(56,78))
rownames(myTable)<-c('Exposure','No exposure')
table2x2(myTable,stats=c('table','or','fisher'))

In [ ]:
%%R
table2x2(myTable,stats=c('table','or','fisher'))

In [ ]:
%%R
install.packages("irr")